In [ ]:
import pandas as pd
import numpy as np
from ggv.utils.context import create_default_context
import warnings
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
context = create_default_context()
bq_db= context.get_ds('YOUR CONTEXT')

In [ ]:
def get_cancel_time(): 

       query= f""" 
       SELECT
  system_order_request_id,
  DATE_DIFF(cancelled_at, created_at, millisecond) AS cancel_time,
  created_at,
  cancelled_at
FROM
  `TableName`
WHERE
  status= 'cancelled'
  AND cancelled_at IS NOT NULL
  AND first_assigned_at IS NULL
GROUP BY
  1,
  2,
  3,
  4
       """ 
       return bq_db.df_from_sql(query)

In [ ]:
def vehicle_cancel(df): 
    df['cancel_minute']= round((df['cancel_time'] / 60000),4)
    df['cancel_second']= round((df['cancel_time'] / 1000),4)
    df2=df.sort_values(by=['cancel_minute']).reset_index(drop=True)
    df2=df.sort_values(by=['cancel_second']).reset_index(drop=True)
    df_cancel_time=df2
    df_cancel_time['count']=1
    df_cancel_time['cum_perc']= 100 * (df_cancel_time['count'].cumsum()) / df_cancel_time['count'].sum() 
    return df_cancel_time 

In [ ]:
data = get_cancel_time()

In [ ]:
data_agg = vehicle_cancel(data)

In [ ]:
def group_cancel_seconds(seconds):
    if seconds <= 10:
        return "0-10"
    elif seconds <= 60:
        return "11-60"
    elif seconds <= 300:
        return "61-300"
    else:
        return "300+"

In [ ]:
def group_cancel_minutes(minutes):
    if minutes <= 1:
        return "0-1"
    elif minutes <= 2:
        return "1-2"
    elif minutes <= 3:
        return "2-3"
    elif minutes <= 4:
        return "3-4"
    elif minutes <= 5:
        return "4-5"
    else:
        return "5+"

In [ ]:
data_agg['cancel_group'] = data_agg['cancel_second'].apply(group_cancel_seconds)

In [ ]:
grouped_data = data_agg.groupby(['cancel_group']).size().reset_index(name='count')
grouped_data = grouped_data.sort_values('count', ascending=True)

In [ ]:
fig = px.bar(grouped_data, 
             x='cancel_group', 
             y='count', 
             barmode='stack',
             labels={
            "cancel_group": "Cancel Group (Seconds)",
                 'count': 'Frequency'
            },
             title='Distribution of Cancel Time (Seconds)'   
)

fig.update_xaxes(categoryorder='array', categoryarray= ['0-10', '11-60','61-300','300+'])

fig.show()

In [ ]:
data_agg['cancel_group'] = data_agg['cancel_minute'].apply(group_cancel_minutes)

In [ ]:
grouped_data = data_agg.groupby(['cancel_group']).size().reset_index(name='count')
grouped_data = grouped_data.sort_values('count', ascending=True)

In [ ]:
fig = px.bar(grouped_data, 
             x='cancel_group', 
             y='count', 
             barmode='stack',
             labels={
            "cancel_group": "Cancel Group (Seconds)",
                 'count': 'Frequency'
            },
             title='Distribution of Cancel Time (Seconds)'   
)

fig.update_xaxes(categoryorder='array', categoryarray= ['0-10', '11-60','61-300','300+'])

fig.show()